In [1]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device, torch_dtype)

cuda torch.float16


In [2]:
# load dataset
dataset = Dataset.load_from_disk("../data/processed/100测试语音/sr_16000.hf")
print(dataset)
print(dataset[0])

df = pd.read_csv("../data/processed/100测试语音/data.csv").drop(columns=["audio_url", "tags", "audio"])
df.head()

Dataset({
    features: ['sentence', 'audio_id', 'audio', 'duration'],
    num_rows: 90
})
{'sentence': '客人青春休闲时尚，喜欢老花，喜欢经典款，喜欢speedy，无锡人，喜欢精致小巧的包型。', 'audio_id': '726157', 'audio': {'path': None, 'array': array([ 0.        ,  0.        ,  0.        , ...,  0.0085144 ,
       -0.00106812, -0.00588989]), 'sampling_rate': 16000}, 'duration': 10.24}


,prediction,sentence,audio_id
0,客人青春休闲时尚，喜欢老花，喜欢春天，还喜欢speed，无数人喜欢精致小巧的包型。,客人青春休闲时尚，喜欢老花，喜欢经典款，喜欢speedy，无锡人，喜欢精致小巧的包型。,726157
1,无锡人刚刚结婚，喜欢休闲的包包，喜欢黑牛角酷一点的，喜欢小邮差包包么teeth。,无锡人刚刚结婚，喜欢休闲的包包，喜欢黑牛角酷一点的，喜欢小邮差包包metis。,726162
2,常州人喜欢小猫，喜欢猫的姑娘，喜欢老婆，喜欢奥那个喜欢戳特。,常州人喜欢小包，喜欢monogram，喜欢老花，喜欢onthego喜欢托特。,726098
3,希望金典款选老华轩爆款需要发票自己开建筑，在附近开鹿，我喜欢大车喜欢经典色。,喜欢经典款，喜欢老花，喜欢爆款，需要发票，自己开店，住在附近开鹿，喜欢大车，喜欢经典色。,725801
4,喜欢休闲，喜欢舒适，喜欢老花，喜欢小刘，才喜欢黑色，喜欢灰色。,喜欢休闲，喜欢舒适，喜欢老花，喜欢小邮差，喜欢黑色，喜欢灰色。,725761


In [3]:
def load_pipeline(hf_model_id: str) -> pipeline:
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        hf_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(hf_model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=4,
        return_timestamps=False,
        torch_dtype=torch_dtype,
        device=device,
    )
    return pipe


def inference_dataset(pipe: pipeline, input_dataset: Dataset) -> list:
    transcriptions = []
    for i in range(len(input_dataset)):
        sample = input_dataset[i]["audio"]
        transcriptions.append(pipe(sample, generate_kwargs={"task": "transcribe", "language": "chinese"}))
    return transcriptions

In [6]:
candidates = {
    "openai/whisper-small": None,
    "openai/whisper-large-v3": None,
    "BELLE-2/Belle-whisper-large-v3-zh": None
}

import os.path as osp
import joblib

if osp.exists("evaluation_100_samples/candidates.pkl"):
    candidates = joblib.load("evaluation_100_samples/candidates.pkl")

else:
    for key in candidates.keys():
        if key != "BELLE-2/Belle-whisper-large-v3-zh":
            _pipe = load_pipeline(key)
            candidates[key] = _pipe(dataset["audio"], generate_kwargs={"task": "transcribe", "language": "chinese"})
        elif key == "BELLE-2/Belle-whisper-large-v3-zh":
            _pipe = pipeline(
                "automatic-speech-recognition",
                model="BELLE-2/Belle-whisper-large-v3-zh",
                max_new_tokens=128,
                chunk_length_s=30,
                batch_size=4,
                return_timestamps=False,
                torch_dtype=torch_dtype,
                device=device,
            )
            _pipe.model.config.forced_decoder_ids = (
                _pipe.tokenizer.get_decoder_prompt_ids(
                    language="zh",
                    task="transcribe"
                )
            )
            candidates[key] = inference_dataset(_pipe, dataset)
        else:
            raise NotImplemented

# save results
joblib.dump(candidates, "evaluation_100_samples/candidates.pkl")

In [14]:
from copy import deepcopy


def extract_text(transcription_dict: dict) -> dict:
    """Extract the transcription text."""
    transcription_dict = deepcopy(transcription_dict)
    for k, v in transcription_dict.items():
        _v = [item["text"] for item in v]
        transcription_dict[k] = _v
    return transcription_dict


candidates = extract_text(candidates)

In [16]:
# add to dataframe
df["openai/whisper-small"] = candidates["openai/whisper-small"]
df["openai/whisper-large-v3"] = candidates["openai/whisper-large-v3"]
df["BELLE-2/Belle-whisper-large-v3-zh"] = candidates["BELLE-2/Belle-whisper-large-v3-zh"]

# convert traditional chinese to simplified chinese
from zhconv import convert


df.to_csv("evaluation_100_samples/data.csv", index=False)

In [17]:
df.head()

,prediction,sentence,audio_id,openai/whisper-small,openai/whisper-large-v3,BELLE-2/Belle-whisper-large-v3-zh
0,客人青春休闲时尚，喜欢老花，喜欢春天，还喜欢speed，无数人喜欢精致小巧的包型。,客人青春休闲时尚，喜欢老花，喜欢经典款，喜欢speedy，无锡人，喜欢精致小巧的包型。,726157,客人青春休閒時尚喜歡老花喜歡金電話喜歡Sbedy無私人喜歡精緻小巧的包行,客人青春休閒時尚喜歡老花喜歡經典款喜歡Speedy無錫人喜歡精緻小巧的包型,客人青春休闲时尚喜欢老花喜欢纪念画喜欢斯比迪无锡人喜欢精致小巧的包型
1,无锡人刚刚结婚，喜欢休闲的包包，喜欢黑牛角酷一点的，喜欢小邮差包包么teeth。,无锡人刚刚结婚，喜欢休闲的包包，喜欢黑牛角酷一点的，喜欢小邮差包包metis。,726162,吳新仁剛剛結婚喜歡休閒的包包喜歡黑牛角褲一點的喜歡小油拆包包Mattice,吴希仁刚刚结婚喜欢休闲的包包喜欢黑牛角酷一点的喜欢小油钗包包MATIS,无锡人刚刚结婚喜欢休闲的包包喜欢黑牛角酷一点的喜欢小邮差包包马蒂斯
2,常州人喜欢小猫，喜欢猫的姑娘，喜欢老婆，喜欢奥那个喜欢戳特。,常州人喜欢小包，喜欢monogram，喜欢老花，喜欢onthego喜欢托特。,726098,常常人喜欢小包 喜欢猫的姑娘 喜欢老婆 喜欢奥特克 喜欢戳特,常州人喜欢小包喜欢蒙多果人喜欢老花喜欢奥德购喜欢托特,常州人喜欢小包喜欢摩托公司喜欢老婆喜欢奥德购喜欢托特
3,希望金典款选老华轩爆款需要发票自己开建筑，在附近开鹿，我喜欢大车喜欢经典色。,喜欢经典款，喜欢老花，喜欢爆款，需要发票，自己开店，住在附近开鹿，喜欢大车，喜欢经典色。,725801,西湾经典款选老华 西湾包款需要发票自己开进住在附近开路我喜欢大车 喜欢经典色,喜欢经典款 喜欢老华 喜欢爆款需要发票自己开进驻在附近开路喜欢大车 喜欢经典色,喜欢经典款喜欢老华喜欢爆款需要发票自己开进住在附近开路喜欢大车喜欢经典色
4,喜欢休闲，喜欢舒适，喜欢老花，喜欢小刘，才喜欢黑色，喜欢灰色。,喜欢休闲，喜欢舒适，喜欢老花，喜欢小邮差，喜欢黑色，喜欢灰色。,725761,喜欢休闲 喜欢舒适 喜欢老花 喜欢雪柔柴 喜欢灰色 喜欢灰色,喜歡休閒喜歡舒適喜歡老花喜歡選柔柴喜歡灰色喜歡灰色,喜欢休闲喜欢舒适喜欢老化喜欢雪柔柴喜欢黑色喜欢灰色


In [ ]:
# ['english'd, 'chinese', 'german', 'spanish', 'russian', 'korean', 'french', 'japanese', 'portuguese', 'turkish', 'polish', 'catalan', 'dutch', 'arabic', 'swedish', 'italian', 'indonesian', 'hindi', 'finnish', 'vietnamese', 'hebrew', 'ukrainian', 'greek', 'malay', 'czech', 'romanian', 'danish', 'hungarian', 'tamil', 'norwegian', 'thai', 'urdu', 'croatian', 'bulgarian', 'lithuanian', 'latin', 'maori', 'malayalam', 'welsh', 'slovak', 'telugu', 'persian', 'latvian', 'bengali', 'serbian', 'azerbaijani', 'slovenian', 'kannada', 'estonian', 'macedonian', 'breton', 'basque', 'icelandic', 'armenian', 'nepali', 'mongolian', 'bosnian', 'kazakh', 'albanian', 'swahili', 'galician', 'marathi', 'punjabi', 'sinhala', 'khmer', 'shona', 'yoruba', 'somali', 'afrikaans', 'occitan', 'georgian', 'belarusian', 'tajik', 'sindhi', 'gujarati', 'amharic', 'yiddish', 'lao', 'uzbek', 'faroese', 'haitian creole', 'pashto', 'turkmen', 'nynorsk', 'maltese', 'sanskrit', 'luxembourgish', 'myanmar', 'tibetan', 'tagalog', 'malagasy', 'assamese', 'tatar', 'hawaiian', 'lingala', 'hausa', 'bashkir', 'javanese', 'sundanese', 'cantonese', 'burmese', 'valencian', 'flemish', 'haitian', 'letzeburgesch', 'pushto', 'panjabi', 'moldavian', 'moldovan', 'sinhalese', 'castilian', 'mandarin']